T5

In [1]:
!pip install -U python-terrier
!pip install -U git+https://github.com/terrierteam/pyterrier_doc2query.git

     |████████████████████████████████| 95 kB 1.9 MB/s 
     |████████████████████████████████| 1.1 MB 37.2 MB/s 
     |████████████████████████████████| 69 kB 5.7 MB/s 
     |████████████████████████████████| 222 kB 42.8 MB/s 
     |████████████████████████████████| 6.3 MB 42.0 MB/s 
     |████████████████████████████████| 294 kB 69.8 MB/s 
     |████████████████████████████████| 72 kB 1.9 MB/s 
     |████████████████████████████████| 1.8 MB 42.4 MB/s 
     |████████████████████████████████| 126 kB 73.9 MB/s 
     |████████████████████████████████| 636 kB 73.9 MB/s 
     |████████████████████████████████| 291 kB 69.3 MB/s 
     |████████████████████████████████| 45 kB 4.3 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.7.0-py3-none-any.whl size=102213 sha256=1ffc26ba252d234e1863770ec72a8cdfa7f6f4c29d87a7d7f8e34e21c602172e
  Stored in directory: /root/.cache/pip/wheels/e3/84/1e/68c08f14e2481e2b3e7c1a2c24bb1220712bc3f5d3896c28df
  Created wheel for ir-measures: filen

In [2]:
import os
import sys
import pyterrier as pt
import pyterrier_doc2query
import pandas as pd

if not pt.started():
    pt.init()

terrier-assemblies 5.6  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6  jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.7.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)


In [3]:
def experiment_br(retrs, topics, qrels, labels):
  return pt.Experiment(retrs, topics, qrels, names=labels, eval_metrics=['map', 'ndcg_cut.10', 'P.10', 'mrt'])

In [4]:
dataset_name = 'irds:vaswani'
path_index_vaswani = '/content/index_vaswani'
props_path = '/data.properties'
dataset = pt.datasets.get_dataset(dataset_name)

if not os.path.exists(path_index_vaswani):
    indexer = pt.index.IterDictIndexer(path_index_vaswani)
    index_ref = indexer.index(dataset.get_corpus_iter(), meta=('docno', 'text'))
else:
    index_ref = pt.IndexRef.of(path_index_vaswani + props_path)

index = pt.IndexFactory.of(index_ref)
topics = dataset.get_topics()
qrels = dataset.get_qrels()
br = pt.BatchRetrieve(index, wmodel='BM25')

[INFO] [starting] building docstore
[INFO] [starting] http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz
docs_iter:   0%|                                     | 0/11429 [00:00<?, ?doc/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 0.0%| 0.00/2.13M [00:00<?, ?B/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 0.8%| 16.4k/2.13M [00:00<00:16, 131kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 2.3%| 49.2k/2.13M [00:00<00:11, 181kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 4.2%| 90.1k/2.13M [00:00<00:09, 219kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 7.3%| 156k/2.13M [00:00<00:07, 280kB/s] 
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 15.0%| 319k/2.13M [00:00<00:03, 456kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 30.5%| 647k/2.13M [00:00<00:01, 767kB/s]

[INFO] [finished] http://ir.dcs.gla.ac.uk/resource

vaswani documents:   0%|          | 0/11429 [00:00<?, ?it/s]

In [5]:
if not os.path.exists('t5-base.zip'):
  !wget https://git.uwaterloo.ca/jimmylin/doc2query-data/raw/master/T5-passage/t5-base.zip
  !unzip t5-base.zip

path_index_t5 = '/content/index_t5'
doc2query = pyterrier_doc2query.Doc2Query(out_attr='text', batch_size=8)
indexer_t5 = doc2query >> pt.IterDictIndexer(path_index_t5)
index_ref_t5 = indexer_t5.index(dataset.get_corpus_iter())
br_doct5 = pt.BatchRetrieve(index_ref_t5, wmodel='BM25')

--2021-09-23 20:58:09--  https://git.uwaterloo.ca/jimmylin/doc2query-data/raw/master/T5-passage/t5-base.zip
Resolving git.uwaterloo.ca (git.uwaterloo.ca)... 129.97.83.4
Connecting to git.uwaterloo.ca (git.uwaterloo.ca)|129.97.83.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 357139559 (341M) [application/zip]
Saving to: ‘t5-base.zip’

t5-base.zip         100%[===================>] 340.59M  40.7MB/s    in 8.8s    

2021-09-23 20:58:22 (38.7 MB/s) - ‘t5-base.zip’ saved [357139559/357139559]

Archive:  t5-base.zip
  inflating: model.ckpt-1004000.data-00000-of-00002  
  inflating: model.ckpt-1004000.data-00001-of-00002  
  inflating: model.ckpt-1004000.index  
  inflating: model.ckpt-1004000.meta  


[INFO] Lock 139664859144848 acquired on /root/.cache/huggingface/transformers/684a47ca6257e4ca71f0037771464c5b323e945fbc58697d2fad8a7dd1a2f8ba.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d.lock


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

[INFO] Lock 139664859144848 released on /root/.cache/huggingface/transformers/684a47ca6257e4ca71f0037771464c5b323e945fbc58697d2fad8a7dd1a2f8ba.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d.lock
[INFO] Lock 139664233125456 acquired on /root/.cache/huggingface/transformers/90de37880b5ff5ac7ab70ff0bd369f207e9b74133fa153c163d14c5bb0116207.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529.lock


Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

[INFO] Lock 139664233125456 released on /root/.cache/huggingface/transformers/90de37880b5ff5ac7ab70ff0bd369f207e9b74133fa153c163d14c5bb0116207.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529.lock
[INFO] Lock 139664226610320 acquired on /root/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.66b9637a52aa11e9285cdd6e668cc0df14b3bcf0b6674cf3ba5353c542649637.lock


Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

[INFO] Lock 139664226610320 released on /root/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.66b9637a52aa11e9285cdd6e668cc0df14b3bcf0b6674cf3ba5353c542649637.lock


Doc2query using cpu


vaswani documents:   0%|          | 0/11429 [00:00<?, ?it/s]

In [6]:
experiment_br([br, br_doct5], topics, qrels, ['BM25', 'T5'])

,name,map,ndcg_cut.10,P.10,mrt
0,BM25,0.296517,0.446609,0.352688,81.268573
1,T5,0.175819,0.320774,0.249462,30.794835


end of fun.